In [1]:
import json
import glob
import os
import datetime
import re
import numpy as np
import pandas as pd
from pathlib import Path
from slack_data_loader2 import SlackLoader2, normalize_links
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.snowball import EnglishStemmer, RussianStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

import eli5
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn import linear_model
from sklearn.pipeline import FeatureUnion
from sklearn.metrics import mean_squared_log_error, roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.grid_search import GridSearchCV

from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score

from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

from operator import itemgetter

%load_ext autoreload
%autoreload 2

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### data
* "channel"
* "start_message_text"
* "thread_text" (__eou__ __eot__)
* "message_type" = ["top", "first_thread", "thread"]
* "attachment" = 1|0
* "message_number" = (1 - top,)
* "message_text" 
### _target_
emoji, count

In [28]:
BASE_DIR = Path("../input/export_Feb_8_2018")

channels = ['_jobs', 'career']

loader = SlackLoader2(BASE_DIR, exclude_channels=[])#, only_channels=channels)
print(len(loader.messages))

#df = pd.DataFrame(loader.threads).set_index(["channel_name", "start_ts"])

751861


In [ ]:
%%time

loader.process_threads()
print(len(loader.threads))
df = pd.DataFrame(loader.threads).set_index(["channel_name", "start_ts"])

  2%|▏         | 12897/751861 [00:01<01:35, 7749.51it/s] 

Empty text {'text_': '', 'text': None, 'attachments': [{'pretext': 'There are no events today.', 'markdwn_in': ['pretext']}], 'channel': 'C04422A5C', 'dt': datetime.datetime(2015, 8, 21, 18, 59, 59, 431), 'type': 'message', 'channel_name': '_meetings', 'subtype': 'bot_message', 'bot_id': 'B09DR8T8S', 'ts': 1440172799.000431}
Empty text {'text_': '', 'text': None, 'attachments': [{'pretext': 'There are no events today.', 'markdwn_in': ['pretext']}], 'channel': 'C04422A5C', 'dt': datetime.datetime(2015, 8, 31, 6, 59, 59, 191), 'type': 'message', 'channel_name': '_meetings', 'subtype': 'bot_message', 'bot_id': 'B09DR8T8S', 'ts': 1440993599.000191}


 19%|█▉        | 146332/751861 [02:21<19:58, 505.24it/s]

In [ ]:
file_cache = \"all_threads.pkl.compress\"\n",
    "#df.to_pickle(file_cache, compression=\"gzip\")\n",

In [ ]:
df.head()

In [29]:
#df = pd.DataFrame(loader.messages)

def add_punkt_nums(df, text_col):
    items = ["words", "commas", "bangs", "quotas"]
    patterns = {"words":'\S+', 
                "commas":'\,',
                "dots":'\.',
                "bangs":'\!',
                "questions":'\!',
                "quotas":'\"', 
                "apost":'\`'
               }
    for name, pat in patterns.items():
        df['num_' + name] = df[text_col].str.count(pat)
    df['avg_word'] = df[text_col].str.len() / (1 + df["num_words"])
    return df

#df = add_punkt_nums(df, "text_")

In [30]:
df.columns

Index(['attachments', 'bot_id', 'bot_link', 'channel', 'channel_id',
       'channel_name', 'client_msg_id', 'comment', 'display_as_bot', 'dt',
       'edited', 'file', 'hidden', 'icons', 'inviter', 'is_intro',
       'is_multiteam', 'is_org_shared', 'is_shared', 'is_thread_broadcast',
       'item', 'item_type', 'members', 'mrkdwn', 'name', 'new_broadcast',
       'no_notifications', 'old_name', 'parent_user_id', 'permalink',
       'plain_text', 'purpose', 'reactions', 'reactions_', 'replies',
       'reply_count', 'room', 'root', 'slog_is_mpdm', 'slog_is_self_dm',
       'slog_is_shared', 'slog_is_slackbot_dm', 'slog_local_team_id',
       'subtype', 'text', 'text_', 'thread_ts', 'timestamp', 'topic', 'ts',
       'type', 'unfurl_links', 'unfurl_media', 'unread_count', 'upload',
       'upload_reply_to', 'user', 'username', 'num_questions', 'num_bangs',
       'num_words', 'num_quotas', 'num_dots', 'num_apost', 'num_commas',
       'avg_word'],
      dtype='object')

In [31]:
#%%time

def get_ktop_reactions_threads(df, k=20):
    df["reactions_"] = df["reactions_"].apply(lambda x : dict(x) )
    df2 = df["reactions_"].apply(pd.Series)
    df3 = pd.concat([df, df2], axis=1).drop('reactions_', axis=1)
    react_stat = df3.drop(["msg_counter"], axis=1).describe().transpose().sort_values(["count"], ascending=False)
    #df_[df_["count"] >= 100] #get by threshold
    cols_to_del = react_stat[k:].index.values
    ktop_cols = react_stat[:k].index.values
    was_rows = df3.shape[0]
    df3 = df3.drop(cols_to_del, axis=1).dropna(axis=0, subset=ktop_cols, how='all')
    print("{} rows droped.".format(was_rows-df3.shape[0]))
    return df3, ktop_cols

def get_ktop_reactions_messages(df, k=20):
    COL_PREFIX = "reaction_"
    df["reactions_"] = df["reactions_"].apply(lambda x : dict(x) )
    df2 = df["reactions_"].apply(pd.Series).add_prefix(COL_PREFIX)
    df3 = pd.concat([df, df2], axis=1).drop('reactions_', axis=1)
    filter_col = [col for col in df3 if col.startswith(COL_PREFIX)]
    react_stat = df3[filter_col].describe().transpose().sort_values(["count"], ascending=False)
    #df_[df_["count"] >= 100] #get by threshold
    cols_to_del = react_stat[k:].index.values
    ktop_cols = react_stat[:k].index.values
    was_rows = df3.shape[0]
    df3 = df3.drop(cols_to_del, axis=1).dropna(axis=0, subset=ktop_cols, how='all')
    print("{} rows droped.".format(was_rows-df3.shape[0]))
    return df3, ktop_cols, react_stat


reacts, ktop_cols, react_stat = get_ktop_reactions_messages(df.dropna(subset=["reactions_"]))



/home/algis/.local/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


13408 rows droped.


In [35]:
reacts.shape

(26808, 85)

In [33]:
ktop_cols

array(['reaction_+1', 'reaction_heavy_plus_sign', 'reaction_joy',
       'reaction_fire', 'reaction_notbad', 'reaction_trollface',
       'reaction_100', 'reaction_povar', 'reaction_muscle',
       'reaction_true-story', 'reaction_yeah-sure', 'reaction_ban',
       'reaction_facepalm', 'reaction_slava', 'reaction_putin',
       'reaction_be-a-man', 'reaction_but_why', 'reaction_kekeke',
       'reaction_wat', 'reaction_ternaus'], dtype=object)

In [36]:
X = reacts['text'].fillna("_na_").values
y = reacts[ktop_cols].fillna(0).values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [43]:
from ufal.udpipe import Model, Pipeline, ProcessingError # pylint: disable=no-name-in-module

In [44]:
def get_pipeline(modelfile='/media/data/word2vec/ufal/udpipe-ud-2.0-170801/russian-syntagrus-ud-2.0-170801.udpipe'):
    model = Model.load(modelfile)
    pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
    return pipeline

pipeline = get_pipeline()

In [47]:
X_train[:30]

array(['&gt; понятно, пойду значит дальше сам писать\nТак ты внимания просто хотел?',
       '<@U04ELQZAU> а можно ссылочку на канал?',
       'мы используем zeppelin + scala при разработке джоб, дебаггинге.\nаналитики и data scientists тоже используют, но они обычно выгружают оттуда данные в jupyter',
       'я однажды в британии сказал барышне из гугла, что она бы неплохо на кухне смотрелась',
       '<https://github.com/deepmind/lab>',
       'Сходила на собеседование в LinkedIn, получила в качестве сувенира распечатанный граф своих контактов. Синий клубок слева - ODS-circlejerk.\n<https://photos.app.goo.gl/JSjyyfi8FBci79HI3>',
       '<https://channel9.msdn.com/Events/Neural-Information-Processing-Systems-Conference/Neural-Information-Processing-Systems-Conference-NIPS-2016/Generative-Adversarial-Networks>',
       'thefuck для Bash: <https://github.com/nvbn/thefuck>',
       'Если будет еще с пяток желающих - я думаю, можно смело создавать.',
       '<https://media.giphy.com/media

In [45]:
processed = pipeline.process()
output = [l for l in processed.split('\n') if not l.startswith('#')]
tagged = ['_'.join(w.split('\t')[2:4]) for w in output if w]

TypeError: in method 'Pipeline_process', argument 2 of type 'std::string const &'

In [37]:
%%time

ru_stemmer = RussianStemmer()
en_stemmer = EnglishStemmer()
analyzer = CountVectorizer().build_analyzer()

def is_russian_word(w):
    #if not isinstance(w, unicode):
    #    w = w.decode('utf8')
    for c in w:
        if ord(c) >= ord(u'а') and ord(c) <= ord(u'я'):
            return True
    return False

def stemmed_words(doc):
    for w in analyzer(doc):
        yield ru_stemmer.stem(w) if is_russian_word(w) else en_stemmer.stem(w)
        

vectorizer = TfidfVectorizer(min_df=5, analyzer=stemmed_words).fit(X_train)        

CPU times: user 31.6 s, sys: 0 ns, total: 31.6 s
Wall time: 31.7 s


### Other vectorizer

In [26]:
%%time

# we need a custom pre-processor to extract correct field,
# but want to also use default scikit-learn preprocessing (e.g. lowercasing)
default_preprocessor = CountVectorizer().build_preprocessor()

import re, string
re_tok = re.compile('([{}“”¨«»®´·º½¾¿¡§£₤‘’])'.format(string.punctuation))
def tokenize(s): return re_tok.sub(r' \1 ', s).split()
  
vectorizer = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1).fit(X_train)


CPU times: user 25.2 s, sys: 172 ms, total: 25.4 s
Wall time: 25.4 s


In [35]:
%%time

vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 5),
    max_features=20000).fit(X_train)

CPU times: user 1min 44s, sys: 556 ms, total: 1min 44s
Wall time: 1min 44s


In [38]:
for _word, _count in sorted(vectorizer.vocabulary_.items(),key=itemgetter(1),reverse=True)[:20]:
    print(_word,end=' ')
    print(_count)

ящик 9990
ячейк 9989
ясн 9988
яростн 9987
ярк 9986
японск 9985
яндекс 9984
январ 9983
ян 9982
якоб 9981
яйц 9980
языков 9979
язык 9978
ядр 9977
ядерн 9976
ядер 9975
явн 9974
явля 9973
явлен 9972
явк 9971


In [ ]:
%%time

def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    param_grid = {'C': Cs, 'dual': [False], 'penalty':['l1','l2'], 'class_weight': [None, 'balanced']}
    #param_grid = {'C': Cs}
    model = LinearSVC(random_state=0)
    grid_search = GridSearchCV(model, param_grid, cv=nfolds, n_jobs=1, scoring='roc_auc', verbose=1)
    grid_search.fit(X, y)
    #grid_search.best_params_
    return grid_search
  

gs_result = svc_param_selection(X_train_vec, y_train, 3)

print(gs_result.best_params_, gs_result.best_score_)

In [39]:
%%time

X_train_vec = vectorizer.transform(X_train)


X_val_vec = vectorizer.transform(X_val)

CPU times: user 34.3 s, sys: 0 ns, total: 34.3 s
Wall time: 34.6 s


In [40]:
def pr(y_i, y, x):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)
  
def get_mdl(X, y):
    #y = y.values
    r = np.log(pr(1,y, X) / pr(0,y, X))
    m = LogisticRegression(C=4, dual=True)
    x_nb = X.multiply(r)
    return m.fit(x_nb, y), r  

def get_rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(np.expm1(y_true), np.expm1(y_pred)))

def get_roc_auc(y_true, y_pred):
    return roc_auc_score(y_true, y_pred)
  
def ridge():
    
    return RidgeClassifier(
        solver='auto',
        fit_intercept=True,
        alpha=0.5,
        max_iter=100,
        normalize=False,
        tol=0.05)




def linear_svm():
    C = 0.01
    return LinearSVC(penalty='l1', dual=False, C=C, random_state=0)

def get_y(y_train, index = 1):
    y_ = y_train[:,0]#.reshape(-1,1)
    return y_ #np.where(y_ > 0, 1, 0)

#X_train_vec = X_train_vec#.toarray()
y_ = get_y(y_train)

cv = KFold(n_splits=10, shuffle=False, random_state=42)
models = []
print("start...")
for train_ids, valid_ids in cv.split(X_train_vec):
    #model = linear_svm()
    #model = linear_model.BayesianRidge()
    model = linear_model.Ridge(alpha = .5)
    #model = linear_model.Lasso(alpha = 0.1)
    #model = linear_model.SGDRegressor()
    #model = linear_model.ElasticNet()
    #model = GradientBoostingRegressor()
    #RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
    model.fit(X_train_vec[train_ids], y_[train_ids])
    y_pred_valid = model.predict(X_train_vec[valid_ids])
    rmsle = get_rmsle(y_pred_valid, y_[valid_ids])
    print('valid rmsle: {}'.format(rmsle))
    models.append(model)
    
print("Make prediction")



test_predicts = np.ones(y_val.shape[0])
for m in models:
    test_predicts *= m.predict(X_val_vec)#.toarray())

test_predicts **= (1. / len(models))

print(get_rmsle(test_predicts, get_y(y_val)))

start...
valid rmsle: 5.587547840901913
valid rmsle: 5.229081737420327
valid rmsle: 4.598652551267773
valid rmsle: 4.895742447573883
valid rmsle: 5.2448290711965
valid rmsle: 5.069583814199927
valid rmsle: 4.963362026046199
valid rmsle: 5.498358699219086
valid rmsle: 4.785797118279256
valid rmsle: 5.521744595744533
Make prediction


/home/algis/.local/lib/python3.5/site-packages/ipykernel/__main__.py:68: RuntimeWarning: invalid value encountered in power


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [41]:
def get_rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(np.expm1(y_true), np.expm1(y_pred)))

def get_ridge():
    return Ridge(
        solver='auto',
        fit_intercept=True,
        alpha=0.5,
        max_iter=100,
        normalize=False,
        tol=0.05)

cv = KFold(n_splits=10, shuffle=True, random_state=42)
for train_ids, valid_ids in cv.split(X_train_vec):
    model = get_ridge()#RandomForestRegressor()
    model.fit(X_train_vec[train_ids], y_[train_ids])
    y_pred_valid = model.predict(X_train_vec[valid_ids])
    rmsle = get_rmsle(y_pred_valid, y_[valid_ids])
    print('valid rmsle: {}'.format(rmsle))
    break


valid rmsle: 5.0328482482568475


In [42]:
eli5.show_weights(model, vec=vectorizer)

Weight?,Feature
+35.932,everyon
+30.570,осен
+27.354,офер
+25.861,pacman
+23.573,стыд
+22.699,hbase
+19.780,speaker
+19.725,druid
+19.268,монстр
+18.731,подписа


In [43]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from scipy.special import logit, expit
from tqdm import tqdm

class_names = ktop_cols

train_text = X_train
test_text = X_val
all_text = np.concatenate([train_text, test_text])

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 1),
    max_features=15000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 5),
    max_features=20000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])


#submission = pd.DataFrame.from_dict(predictions)
#submission.to_csv('submission.csv', index=False)

NameError: name 'test' is not defined

In [ ]:
losses = []
#predictions = {'id': test['id']}

#y = reacts[ktop_cols].fillna(0).values


def get_ridge():
    return Ridge(
        solver='auto',
        fit_intercept=True,
        alpha=0.5,
        max_iter=100,
        normalize=False,
        tol=0.05)

for index, class_name in tqdm(enumerate(class_names)):
    #train_target = np.where(y_train[:,index] > 0, 1,0)
    train_target = y_train[:,index]
    #classifier = get_ridge()#LogisticRegression(solver='sag')
    classifier = linear_model.Ridge(alpha = .5)
    cv_loss = np.mean(cross_val_score(classifier, train_features, train_target, cv=5, scoring='neg_mean_squared_error'))
    losses.append(cv_loss)
    print('CV score for class {} is {}'.format(class_name, cv_loss))

    #classifier.fit(train_features, train_target)
    #predictions[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(losses)))



0it [00:00, ?it/s]
